In [ ]:
pip install pyspark 

In [ ]:
pip install numpy 

In [16]:
import os
from pyspark.sql import SparkSession

# Create a Spark session
os.environ["HADOOP_HOME"] = "C:\\hadoop-3.3.5"

# Set hadoop.home.dir system property
os.environ["hadoop.home.dir"] = "C:\\hadoop-3.3.5"

# Initialize SparkSession
spark = SparkSession.builder.appName("ebay_analysis").getOrCreate()

# File location and type
file_location = "Data\\Final_Dataset.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Read data from CSV file into a DataFrame
final_dataset = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

# Show the DataFrame
final_dataset.show()

+---+---------------+--------------------+----------+------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+--------------------+--------------------+
|PID|       Category|            Location|  Latitude|   Longitude|Gender|               Title|Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|   Product Condition|                 URL|
+---+---------------+--------------------+----------+------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+--------------------+--------------------+
|  0|Car Accessories|Boston, United St...|42.3554334|  -71.060511|Unisex|Car Steering Whee...|15.46|               2|                    3|   3.0|             luobo-e|           100%|                 New|https://www.ebay....|
|  1|Car Accessories|Chicago, Illinois...|41.8755616| -87.6244212|Unisex|1pc Carbon Fiber ...| 6

In [5]:
final_dataset.printSchema()

root
 |-- PID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Total Sold Items: string (nullable = true)
 |-- Total Available Items: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Seller Name: string (nullable = true)
 |-- Seller Feedback: string (nullable = true)
 |-- Product Condition: string (nullable = true)
 |-- URL: string (nullable = true)


In [3]:
final_dataset = final_dataset.dropna()

In [6]:
from pyspark.sql.functions import col

final_dataset = final_dataset.withColumn("Price", col("Price").cast("double"))
final_dataset = final_dataset.withColumn("Total Sold Items", col("Total Sold Items").cast("int"))
final_dataset = final_dataset.withColumn("Total Available Items", col("Total Available Items").cast("int"))
final_dataset = final_dataset.withColumn("Rating", col("Rating").cast("double"))


In [7]:
final_dataset = final_dataset.drop("URL")


In [8]:
# Drop duplicate rows based on selected columns
columns_for_deduplication = ['PID', 'Category', 'Location', 'Latitude', 'Longitude', 'Seller Name', 'Title']
final_dataset = final_dataset.dropDuplicates(subset=columns_for_deduplication)
final_dataset.show()

+-----------+--------------------+--------------------+------------------+------------------+-----------+--------------------+-----+----------------+---------------------+------+--------------------+--------------------+-----------------+
|        PID|            Category|            Location|          Latitude|         Longitude|     Gender|               Title|Price|Total Sold Items|Total Available Items|Rating|         Seller Name|     Seller Feedback|Product Condition|
+-----------+--------------------+--------------------+------------------+------------------+-----------+--------------------+-----+----------------+---------------------+------+--------------------+--------------------+-----------------+
|42949700571|https://www.ebay....|      """Minnesota""|    United States"|        43.6166163|-116.200886|              Unisex| 24.0|               9|                  755|  NULL|                 5.0|RBP Random Bike P...|            99.8%|
|25769856955|              Sports|    ., Uni

In [9]:
from pyspark.sql.functions import regexp_replace

final_dataset = final_dataset.withColumn("Title", regexp_replace(col("Title"), "[^a-zA-Z0-9\\s]", ""))


In [10]:
# Remove rows where Category contains URLs
final_dataset = final_dataset.filter(~col('Category').contains('http'))


In [9]:
final_dataset.show()

+-----------+--------------------+--------------------+------------------+------------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+-----------------+
|        PID|            Category|            Location|          Latitude|         Longitude|Gender|               Title|Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|Product Condition|
+-----------+--------------------+--------------------+------------------+------------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+-----------------+
|25769856955|              Sports|    ., United States|        39.7837304|       -100.445882|  Male|Basketball Backpa...|46.69|               1|                    5|   2.5|        fierex-deals|           100%|    New with tags|
|25769804259|Home and Industri...|    ., United States|        39.7837304|       -10

In [11]:
# Convert Rating to DoubleType
from pyspark.sql.types import DoubleType


final_dataset = final_dataset.withColumn('Rating', col('Rating').cast(DoubleType()))

final_dataset.show()

+-----------+--------------------+--------------------+------------------+------------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+-----------------+
|        PID|            Category|            Location|          Latitude|         Longitude|Gender|               Title|Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|Product Condition|
+-----------+--------------------+--------------------+------------------+------------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+-----------------+
|25769856955|              Sports|    ., United States|        39.7837304|       -100.445882|  Male|Basketball Backpa...|46.69|               1|                    5|   2.5|        fierex-deals|           100%|    New with tags|
|25769804259|Home and Industri...|    ., United States|        39.7837304|       -10

In [12]:
# Drop duplicate rows based on the 'Title' column
final_dataset = final_dataset.dropDuplicates(subset=['Title'])

final_dataset.show()

+-----------+--------------------+--------------------+----------+------------+------+--------------------+-------+----------------+---------------------+------+--------------------+---------------+-----------------+
|        PID|            Category|            Location|  Latitude|   Longitude|Gender|               Title|  Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|Product Condition|
+-----------+--------------------+--------------------+----------+------------+------+--------------------+-------+----------------+---------------------+------+--------------------+---------------+-----------------+
|17179912701|Home and Industri...|Fort Payne, Alaba...|34.4442547| -85.7196893|Unisex| 12 Pair Toddler ...|   8.49|              12|                   10|   4.0|             pam6209|          99.5%|    New with tags|
|25769815441|            Vehicles|New York, New Yor...|40.7127281| -74.0060152|Unisex|   1985 HARLEY FXRT |16500.0|              36|

In [13]:
final_dataset.printSchema()


root
 |-- PID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Total Sold Items: integer (nullable = true)
 |-- Total Available Items: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Seller Name: string (nullable = true)
 |-- Seller Feedback: string (nullable = true)
 |-- Product Condition: string (nullable = true)


In [14]:
final_dataset.coalesce(1).write.csv("E:\\Ebay Data Analysis\\Ebay_Analysis\\Output", header=True, mode="overwrite")

In [12]:
temp_df = final_dataset
temp_df.show()

+-----------+--------------------+--------------------+----------+------------+------+--------------------+-------+----------------+---------------------+------+--------------------+---------------+-----------------+
|        PID|            Category|            Location|  Latitude|   Longitude|Gender|               Title|  Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|Product Condition|
+-----------+--------------------+--------------------+----------+------------+------+--------------------+-------+----------------+---------------------+------+--------------------+---------------+-----------------+
|17179912701|Home and Industri...|Fort Payne, Alaba...|34.4442547| -85.7196893|Unisex| 12 Pair Toddler ...|   8.49|              12|                   10|   4.0|             pam6209|          99.5%|    New with tags|
|25769815441|            Vehicles|New York, New Yor...|40.7127281| -74.0060152|Unisex|   1985 HARLEY FXRT |16500.0|              36|

In [45]:
# Top selling categories in each location 

from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window

# Add a rank to each category based on the total sold items
windowSpec = Window.partitionBy("Location").orderBy(desc("Total Sold Items"))

top_categories_in_each_location = (
    temp_df
    .withColumn("rank", rank().over(windowSpec))
    .filter(col("rank") <= 3)  # Adjust the number as needed
    .select("Location","Latitude", "Longitude", "Category", "Total Sold Items")
    .orderBy("Location", "rank")
)

top_categories_in_each_location.show(20, truncate=False)

# top_categories_in_each_location.coalesce(1).write.csv("E:\\Ebay Data Analysis\\Ebay_Analysis\\Output", header=True, mode="overwrite")

# top_categories_in_each_location.printSchema()


+-----------------------------------------------------------+------------------+-------------------+-------------------------------+----------------+
|Location                                                   |Latitude          |Longitude          |Category                       |Total Sold Items|
+-----------------------------------------------------------+------------------+-------------------+-------------------------------+----------------+
|, Oxfordshire, United Kingdom                              |51.833333         |-1.25              |Electronics                    |26              |
|., United States                                           |39.7837304        |-100.445882        |Sports                         |1               |
|., United States                                           |39.7837304        |-100.445882        |Home and Industrial Accessories|1               |
|06001, United States                                       |41.791279959227765|-72.8667468642877  |

In [50]:
#Top selling locations 

from pyspark.sql.functions import sum

top_selling_locations = (
    temp_df
    .groupBy("Location", "Latitude", "Longitude")
    .agg(sum("Total Sold Items").alias("Total Items Sold"))
    .orderBy(desc("Total Items Sold"))
)

top_selling_locations.show(truncate=False)
top_selling_locations.coalesce(1).write.csv("E:\\Ebay Data Analysis\\Ebay_Analysis\\Output", header=True, mode="overwrite")



+-------------------------------------------+----------+------------+----------------+
|Location                                   |Latitude  |Longitude   |Total Items Sold|
+-------------------------------------------+----------+------------+----------------+
|New York, New York, United States          |40.7127281|-74.0060152 |310671          |
|Chicago, Illinois, United States           |41.8755616|-87.6244212 |293611          |
|Dumont, New Jersey, United States          |40.9406541|-73.9968051 |268651          |
|Longwood, Florida, United States           |28.7008496|-81.3465575 |268362          |
|Boston, United States                      |42.3554334|-71.060511  |264147          |
|Houston, TX, United States                 |29.7589382|-95.3676974 |263052          |
|Redwood City, California, United States    |37.4863239|-122.232523 |260538          |
|Ontario, California, United States         |34.065846 |-117.6484304|181444          |
|Los Angeles, California, United States    

In [51]:
# Geographic distribution of sellers by latitude and longitude
from pyspark.sql.functions import countDistinct

seller_geographic_spread = (
    temp_df
    .groupBy("Seller Name", "Latitude", "Longitude")
    .agg(countDistinct("Location").alias("Unique Locations"))
    .filter(col("Unique Locations") > 1)  # Sellers with presence in multiple locations
    .orderBy(desc("Unique Locations"))
)

seller_geographic_spread.printSchema()
# seller_geographic_spread.show(truncate=False)
seller_geographic_spread.coalesce(1).write.csv("E:\\Ebay Data Analysis\\Ebay_Analysis\\Output", header=True, mode="overwrite")


root
 |-- Seller Name: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Unique Locations: long (nullable = false)


In [54]:
#Price Distribution by location 
from pyspark.sql.functions import avg, stddev
price_distribution_by_location = (
    temp_df
    .groupBy("Location")
    .agg(avg("Price").alias("Average Price"), stddev("Price").alias("Price Standard Deviation"))
)

# price_distribution_by_location.show(truncate=False)
price_distribution_by_location.printSchema()
price_distribution_by_location.coalesce(1).write.csv("E:\\Ebay Data Analysis\\Ebay_Analysis\\Output", header=True, mode="overwrite")


root
 |-- Location: string (nullable = true)
 |-- Average Price: double (nullable = true)
 |-- Price Standard Deviation: double (nullable = true)


In [34]:
from pyspark.sql.functions import col, sum, desc, split

# Assuming your dataframe is named 'final_dataset'
# Assuming you have a 'Location' column with state names

# Define a function to extract state from the 'Location' column
def extract_state(location_col):
    # Split the 'Location' column by commas and get the second element (assuming state is the second element)
    return split(location_col, ',')[1].trim()

# Create a dictionary to map states to regions
state_to_region = {
    'California': 'West Coast',
    'Oregon': 'West Coast',
    'Washington': 'West Coast',
    'Nevada': 'West Coast',
    'New York': 'East Coast',
    'New Jersey': 'East Coast',
    'Connecticut': 'East Coast',
    'Florida' : 'East Coast',
    'Georgia' : 'East Coast'
}

# Define a function to determine the region based on state
def determine_region(state_col):
    return state_to_region.get(state_col, 'Other')

# Create a new column 'State' by extracting it from the 'Location' column
temp_df_state = final_dataset.withColumn("State", extract_state(col("Location")))

# Create a new column 'Region' based on the 'State' column
temp_df_coast = temp_df_state.withColumn("Region", determine_region(col("State")))

# Group by 'Region' and 'Category' and calculate the total sold items
popular_categories_by_region = (
    temp_df_coast
    .groupBy("Region", "Category")
    .agg(sum(col("Total Sold Items")).alias("Total Sold Items"))
    .orderBy("Region", desc("Total Sold Items"))
)

# Show the results
popular_categories_by_region.show(truncate=False)

TypeError: 'Column' object is not callable

In [40]:
from pyspark.sql.functions import col, count, desc

# Assuming your dataframe is named 'final_dataset'

# Group by 'Category' and 'Gender', then calculate the count of each category for each gender
category_analysis_by_gender = (
    temp_df
    .groupBy("Category", "Gender")
    .agg(count("*").alias("Count"))
    .orderBy("Category", "Gender")
)

# Show the results
category_analysis_by_gender.show(truncate=False)


+-------------------------------+------+-----+
|Category                       |Gender|Count|
+-------------------------------+------+-----+
|17                             |Unisex|1    |
|4.8                            |Unisex|1    |
|Antiques                       |Female|489  |
|Antiques                       |Male  |2519 |
|Antiques                       |Unisex|1947 |
|Automotive                     |Male  |438  |
|Automotive                     |Unisex|18354|
|Car Accessories                |Female|5    |
|Car Accessories                |Male  |121  |
|Car Accessories                |Unisex|6732 |
|Electronics                    |Female|676  |
|Electronics                    |Male  |5893 |
|Electronics                    |Unisex|55940|
|For parts or not working       |Male  |1    |
|For parts or not working       |Unisex|2    |
|Home and Industrial Accessories|Female|131  |
|Home and Industrial Accessories|Male  |348  |
|Home and Industrial Accessories|Unisex|15606|
|Kids        

In [3]:
file_location1 = "Data\\Cleanest_Dataset_Deduplication.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Read data from CSV file into a DataFrame
clean_df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

# Show the DataFrame
clean_df.show()

+---+---------------+--------------------+----------+------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+--------------------+--------------------+
|PID|       Category|            Location|  Latitude|   Longitude|Gender|               Title|Price|Total Sold Items|Total Available Items|Rating|         Seller Name|Seller Feedback|   Product Condition|                 URL|
+---+---------------+--------------------+----------+------------+------+--------------------+-----+----------------+---------------------+------+--------------------+---------------+--------------------+--------------------+
|  0|Car Accessories|Boston, United St...|42.3554334|  -71.060511|Unisex|Car Steering Whee...|15.46|               2|                    3|   3.0|             luobo-e|           100%|                 New|https://www.ebay....|
|  1|Car Accessories|Chicago, Illinois...|41.8755616| -87.6244212|Unisex|1pc Carbon Fiber ...| 6

In [4]:
clean_df.printSchema()

root
 |-- PID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Total Sold Items: string (nullable = true)
 |-- Total Available Items: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Seller Name: string (nullable = true)
 |-- Seller Feedback: string (nullable = true)
 |-- Product Condition: string (nullable = true)
 |-- URL: string (nullable = true)


In [ ]:
category_location_df = category_location_df.withColumn("Total Sold Items", col("Total Sold Items").cast("int"))
